# Argument Quality model:
This model takes in an argument and outputs a score on its quality.



## Import Libraries

In [ ]:
! pip install evaluate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [ ]:
import pandas as pd
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, AutoConfig, AdamW, get_scheduler
import numpy as np
import evaluate
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn as nn

In [ ]:
df = pd.read_csv('arg_quality_rank_30k.csv')
df.head()

,argument,topic,set,WA,MACE-P,stance_WA,stance_WA_conf
0,"""marriage"" isn't keeping up with the times. a...",We should abandon marriage,train,0.846165,0.297659,1,1.000000
1,.a multi-party system would be too confusing a...,We should adopt a multi-party system,train,0.891271,0.726133,-1,1.000000
2,\ero-tolerance policy in schools should not be...,We should adopt a zero-tolerance policy in sch...,dev,0.721192,0.396953,-1,1.000000
3,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,train,0.730395,0.225212,-1,1.000000
4,"100% agree, should they do that, it would be a...",We should abolish safe spaces,train,0.236686,0.004104,1,0.805517


In [ ]:
df.iloc[4, 0]

'100% agree, should they do that, it would be a good thing'

Initialize model and tokenizer using BERT

split into train and testing sets

In [ ]:
train_set = df[df['set'] == 'train']
test_set = df[df['set'] == 'test']

In [ ]:
pretrained_model = 'bert-base-uncased'
config = AutoConfig.from_pretrained(pretrained_model, num_labels=1, problem_type="regression") # Chose regression because the WA values are continuous in the dataset
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# ensuring it runs on GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

cuda


In [ ]:
# Tokenize the arguments
train_encodings = tokenizer(train_set['argument'].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_set['argument'].tolist(), truncation=True, padding=True, max_length=128)

# Convert labels to tensor
train_labels = train_set['WA'].tolist()
test_labels = test_set['WA'].tolist()

In [ ]:
from torch.utils.data import Dataset, DataLoader
class ArgQDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ArgQDataset(train_encodings, train_labels)
test_dataset = ArgQDataset(test_encodings, test_labels)

In [ ]:
from sklearn.metrics import mean_squared_error

def compute_metrics(p):
    preds = p.predictions.squeeze()
    mse = mean_squared_error(p.label_ids, preds)
    return {'mse': mse}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

eval_results = trainer.evaluate()
print(f"Mean Squared Error: {eval_results['eval_mse']}")



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse
1,0.024500,0.036027,0.036027
2,0.023200,0.032898,0.032898
3,0.010900,0.037062,0.037062


Mean Squared Error: 0.03706160932779312


In [ ]:
trainer.save_model('/content/argument_quality_analyzer')

In [ ]:
# Function to predict argument quality
def predict_argument_quality(argument):
    # Tokenize the input argument
    inputs = tokenizer(argument, return_tensors='pt', truncation=True, padding=True, max_length=128)

    # Move inputs to the same device as the model
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU

    # Set the model to evaluation mode
    model.eval()

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        score = outputs.logits.squeeze().item()

    # Clamp the score between 0 and 1
    clamped_score = max(0.0, min(1.0, score))

    return clamped_score

# Example usage
strong_argument = "A strong economy is essential for the well-being of a nation. When the economy is robust, there are more job opportunities, higher wages, and increased consumer spending, all of which contribute to a higher standard of living. Additionally, a strong economy allows the government to invest in critical infrastructure and social services, such as healthcare and education, which further improve the quality of life for citizens. Historical data shows that countries with strong economies experience lower crime rates and higher levels of happiness among their populations. Therefore, policies that promote economic growth should be a top priority for any government."

weak_argument = "They should do that, its good"
strong_score = predict_argument_quality(strong_argument)
weak_score = predict_argument_quality(weak_argument)

print(f"Strong Argument Predicted Quality Score: {strong_score}")
print(f"Weak Argument Predicted Quality Score: {weak_score}")

Strong Argument Predicted Quality Score: 1.0
Weak Argument Predicted Quality Score: 0.22027210891246796


In [ ]:
!zip -r file.zip /content/argument_quality_analyzer
from google.colab import files
files.download('file.zip')

  adding: content/argument_quality_analyzer/ (stored 0%)
  adding: content/argument_quality_analyzer/training_args.bin (deflated 51%)
  adding: content/argument_quality_analyzer/config.json (deflated 48%)
  adding: content/argument_quality_analyzer/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>